In [1]:
import hail as hl
from pcgc_hail.hail_scripts.utils import *
from pcgc_hail.hail_scripts.annotate_frequencies import *

mt = hl.read_matrix_table('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/gruber_WES_b5_vep_filtered_FINAL.mt')
mt.describe()

Initializing Hail with default parameters...


2021-09-23 15:48:25 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-09-23 15:48:26 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.1.2
SparkUI available at http://c20n06.ruddle.hpc.yale.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.74-0c3a74d12093
LOGGING: writing to /gpfs/ycga/scratch60/lek/ml2529/gruber_analysis/hail-20210923-1548-0.2.74-0c3a74d12093.log


----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSITIVE_TRAIN_SITE: bool, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        VQSLOD: float64, 
        culprit: str
    }
    'allele_data': struct {
        nonsplit_alleles: array<str>, 
     

In [4]:
# sample information 
file_input = 'sample_groups.txt'
print('Reads in the following txt as a hail table: \n' + file_input + '\n')
ht = hl.import_table(file_input).key_by('Adult-ID')
ht = ht.rename({'Adult-ID' : 's'}) #to match mt

# add phenoype group as column annotation
mt = mt.annotate_entries(phenotype = ht[mt.s].group)
mt.describe()


# annotate with AC_adj for each phenotype group
mt = mt.annotate_rows(AC_adj_Barlow = hl.agg.filter((mt.phenotype == "Barlow") & mt.adj, hl.agg.call_stats(mt.GT, mt.alleles).AC[1]))
mt = mt.annotate_rows(AC_adj_FED = hl.agg.filter((mt.phenotype == "FED") & mt.adj, hl.agg.call_stats(mt.GT, mt.alleles).AC[1]))



misD_mt = mt.filter_rows((mt.gnomad_af <= 0.001) &
                             (mt.variant_qc.gq_stats.mean > 98) &
                             (mt.AC_adj < 10) &
                             (mt.category == "missense") &
                            (mt.meta_svm == "D"))

misD_ht = (misD_mt.group_rows_by(misD_mt.gene_symbol)
                          .aggregate_rows(gene_count = hl.agg.sum(misD_mt.AC_adj))
                          .result()).rows()

#misD_ht.show()
#misD_ht.export('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/misD_gene_counts.tsv')

Reads in the following txt as a hail table: 
sample_groups.txt

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        NEGATIVE_TRAIN_SITE: bool, 
        POSITIVE_TRAIN_SITE: bool, 
        QD: float64, 
        RAW_MQandDP: array<int32>, 
        ReadPosRankSum: float64, 
        SOR: float64, 
        VQSLOD: float64, 
        culprit: str
    }
    'all

2021-09-23 15:54:09 Hail: INFO: Reading table without type imputation
  Loading field 'Adult-ID' as type str (not specified)
  Loading field 'Age_years' as type str (not specified)
  Loading field 'group' as type str (not specified)
2021-09-23 15:54:09 Hail: WARN: 'GroupedMatrixTable.result': No entry fields were defined.


In [5]:

misD_BD_ht = (misD_mt.group_rows_by(misD_mt.gene_symbol)
                          .aggregate_rows(gene_count = hl.agg.sum(misD_mt.AC_adj_Barlow))
                          .result()).rows()

#misD_ht.show()

misD_BD_ht.export('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/misD_BD_gene_counts.tsv')



misD_FED_ht = (misD_mt.group_rows_by(misD_mt.gene_symbol)
                          .aggregate_rows(gene_count = hl.agg.sum(misD_mt.AC_adj_FED))
                          .result()).rows()


misD_FED_ht.export('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/misD_FED_gene_counts.tsv')



2021-09-23 15:54:24 Hail: WARN: 'GroupedMatrixTable.result': No entry fields were defined.
2021-09-23 15:54:31 Hail: INFO: Ordering unsorted dataset with network shuffle0]
2021-09-23 15:54:31 Hail: INFO: merging 30 files totalling 26.9K...
2021-09-23 15:54:31 Hail: INFO: while writing:
    /gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/misD_BD_gene_counts.tsv
  merge time: 16.866ms
2021-09-23 15:54:32 Hail: WARN: 'GroupedMatrixTable.result': No entry fields were defined.
2021-09-23 15:54:34 Hail: INFO: Ordering unsorted dataset with network shuffle0]
2021-09-23 15:54:34 Hail: INFO: merging 30 files totalling 26.9K...
2021-09-23 15:54:34 Hail: INFO: while writing:
    /gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/misD_FED_gene_counts.tsv
  merge time: 13.577ms


In [19]:

lof_mt = mt.filter_rows((mt.gnomad_af <= 0.001) &
                             (mt.AC_adj < 5) &
                             (mt.category == "lof") &
                             (mt.variant_class == "SNV"))

'''
lof_mt = mt.filter_rows((mt.gnomad_af <= 0.001) &
                             (mt.AC_adj < 5) &
                             (mt.category == "lof"))

'''

lof_ht = (lof_mt.group_rows_by(lof_mt.gene_symbol)
                          .aggregate_rows(gene_count = hl.agg.sum(lof_mt.AC_adj))
                          .result()).rows()

lof_ht.show()

2021-09-20 14:23:19 Hail: WARN: 'GroupedMatrixTable.result': No entry fields were defined.
2021-09-20 14:23:19 Hail: INFO: Ordering unsorted dataset with network shuffle


,
gene_symbol,gene_count
str,int64
"""A2ML1""",4
"""AARSD1""",1
"""AATF""",1
"""AATK""",1
"""ABC13-47656900H15.1""",2
"""ABCA1""",1
"""ABCA12""",1
"""ABCA13""",1


In [20]:
lof_ht.export('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/lof_gene_counts.tsv')

2021-09-20 14:23:25 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-20 14:23:25 Hail: INFO: merging 30 files totalling 18.7K...
2021-09-20 14:23:25 Hail: INFO: while writing:
    /gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/lof_gene_counts.tsv
  merge time: 29.797ms
